# **HW 4: Time Series Analysis**

# **Instructions**

# **Tasks**

Data Retrieval
Use the Yahoo Finance API to download NVDA daily price data.

Data Export
Save the NVDA daily price data as a CSV file using the to_csv function in Pandas and download it to your local machine.

Time Series Modeling in Excel
Build a time series model in Excel to predict NVDA’s daily price for January and February 2025.

Prediction Accuracy
Evaluate the model’s performance using Mean Squared Error (MSE).

Residual Analysis
Analyze the prediction errors. Identify specific dates where the errors are unusually high. Research possible external factors (e.g., news, reports, product releases) that may explain these anomalies.

Forecasting with Prophet
In a Jupyter Notebook, use the Prophet library to forecast NVDA’s daily price for January and February 2025.

Comparison & Residual Analysis
Compare the predictions from Excel and Prophet. Perform a similar residual analysis for the Prophet model. Are there more data points with large errors in the Prophet version?

# **Submission**

Submit your Excel file containing the time series model.
•
Submit your Jupyter Notebook with the Prophet-based forecast and analysis.

In [ ]:
import yfinance as yf
import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

# Step 1: Download and prepare data
ticker = "NVDA"
start_date = "2010-01-01"
end_date = "2025-02-28"

data = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False)
data = data.reset_index()

# Fill in all calendar dates, forward-fill stock values - Required for Excel Time Series Analysis
full_dates = pd.date_range(start=data['Date'].min(), end=data['Date'].max())
data = data.set_index('Date').reindex(full_dates).rename_axis('Date').reset_index()
data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']] = data[['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']].ffill()

# Save for Excel use
data.to_csv('NVDA_daily_price_data1.csv', index=False)

# Step 2: Prepare for Prophet
prophet_data = data[['Date', 'Close']].copy()
prophet_data.columns = ['ds', 'y']
prophet_data = prophet_data.dropna()

# Step 3: Train Prophet model
model = Prophet(daily_seasonality=True)
model.fit(prophet_data)

# Step 4: Forecast next 60 days
future = model.make_future_dataframe(periods=60)
forecast = model.predict(future)

# Step 5: Merge forecast with actuals
merged = pd.merge(forecast[['ds', 'yhat']], prophet_data, on='ds', how='inner')

# Step 6: Filter for Jan–Feb 2025
jan_feb = merged[(merged['ds'] >= '2025-01-01') & (merged['ds'] < '2025-03-01')].copy()
jan_feb.dropna(inplace=True)

# Step 7: Calculate residuals and MSE
jan_feb['residual'] = jan_feb['y'] - jan_feb['yhat']
mse = mean_squared_error(jan_feb['y'], jan_feb['yhat'])

# Step 8: Plot forecast vs actual with labels
plt.figure(figsize=(12, 6))
plt.plot(jan_feb['ds'], jan_feb['y'], label='Actual', marker='o')
plt.plot(jan_feb['ds'], jan_feb['yhat'], label='Forecast', linestyle='--', marker='x')
plt.title('NVDA Forecast vs Actual (Jan–Feb 2025)')
plt.xlabel('Date')
plt.ylabel('Price')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

# Step 9: Plot residuals
plt.figure(figsize=(12, 4))
plt.bar(jan_feb['ds'], jan_feb['residual'])
plt.axhline(0, color='red', linestyle='--')
plt.title('Residuals (Actual - Forecast)')
plt.xlabel('Date')
plt.ylabel('Residual')
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(True)
plt.show()

# Step 10: Model performance summary
print("📊 MODEL PERFORMANCE SUMMARY")
print(f"Mean Squared Error (Jan–Feb 2025): {round(mse, 4)}")

# Step 11: Show forecast vs actual per day + residual
print("\n📅 FORECAST VS ACTUAL + RESIDUALS (Jan–Feb 2025):")
print(jan_feb[['ds', 'yhat', 'y', 'residual']].rename(columns={
    'ds': 'Date', 'yhat': 'Forecast', 'y': 'Actual', 'residual': 'Residual'
}).to_string(index=False))


# **Excel Residual Analysis**

The Excel MSE is 63.362

Top three dates with highest residuals


1.   01/06/2025
  *  NVIDIA CEO Jensen Huang delivered the opening keynote for CES, which is one of the worlds largest consumer electronics trade shows. In his keynote Jensen revealed several new project lines  and partnerships which can explain the variance in are actual vs predicted values. Examples of new products and partships include the NVIDIA Cosmos, New NVIDIA Blackwell-based GeForce RTX 50 Series GPUs, The new NVIDIA Project DIGITS and lastly NVIDIA is partnering with Toyota for safe next-gen vehicle development.
  
  https://blogs.nvidia.com/blog/ces-2025-jensen-huang/

  https://www.forbes.com/sites/digital-assets/2025/01/08/9-nvidia-announcements-from-ces-2025-and-their-impact-on-blockchain/

2.   01/23/2025

  *   NVIDIA stock was riding the wave from previous days. See the analysis below as it is the same.





3.   01/22/2025
  *   Nvidia announced the Stargate project which is a new joint company formed by OpenAI, Oracle, and Softbank. The three partners have announced they will soon be investing 100 billion in U.S. AI infrastructure. They also have the backing of United States Government as President Trump hosted all three CEO's and Chairman's to announce the Stargate's AI infrastrucutre initiative. This venture will focus on expanding AI-focused data centers domestically. While the project will initially invest 100 billion however over next four years they could potentially invest 500 billion.

  https://www.fool.com/investing/2025/01/22/why-nvidia-stock-surged-today/
  https://www.wsj.com/livecoverage/stock-market-today-dow-sp500-nasdaq-live-01-22-2025/card/zsuKorU4YWdWorMl2D45





# **Prophet Residual Analysis**

The Python Prophet MSE is 200.6044

The daily residuals for the Prophet model were much higher than the excel model, which is reflected in the larger MSE. The largest, however, residuals occurred on the same dates as the excel model on 1/6, 1/22, and 1/23. Refer to the above residual analysis on the excel model for the likely explanation of the large residuals on those dates.

# **Excel & Prophet Comparison**

Python Prophet MSE = 200.6044

Excel MSE = 63.362


While both models forecasted NVDA's price movement from Jan–Feb 2025, the Excel model significantly outperformed the Prophet model in terms of Mean Squared Error (63.36 vs 200.60).

Possible Reasons for discrepency comparison:

**Excel Model:** Potentially overfitted or more reactive to recent trends, it might have captured short-term anomalies (like CES announcements or government-backed AI initiatives) better. This helped reduce its residuals for key event dates.

**Prophet Model:** Designed for general trend and seasonality detection, it smooths out sudden spikes or dips that don't align with historical seasonality. As a result, it missed capturing sharp rises caused by sudden news or partnerships.

In short, Prophet is better for long-term trend forecasting, but Excel may excel in short-term, event-driven predictions—especially when recent price patterns strongly influence future values.